In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
# from data_prep.py import 

# Modelsetup (datamerge)


This notebook uses the following datasets to merge ep-online(BAG) data with klimaatmonitordata:
1. BAG data per house, including buildingtypes, retrieved from: [eponline](https://ep-online.nl/ep-online/) 
2. Energy data per buurt, retrieved from: [klimaatmonitor](https://klimaatmonitor.databank.nl/Jive)
3. [Solar PV data per neighbourhood (CBS)](https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS&tableId=84517NED&_theme=287)
4. on chargingpoints (private)
5. [FEV and PHEV data on low scale](https://klimaatmonitor.databank.nl/Jive?workspace_guid=b341f460-f6d7-4c47-b74e-808f0e32e096)
6. [CBS data on vehicles per zipcode](https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS&tableId=37209hvv&_theme=439)
7. [Green gas and renewable heat per municipality](https://klimaatmonitor.databank.nl/Jive?workspace_guid=84d4a5ab-b92e-4c0d-a293-16215c6d899c)

<br>
@author: Mark Hupkens
last edit: 13-05-2019

All code available on [github repo](https://github.com/markhupkens/EnergyTransitionModelling)

> __To do (21-05):__
1. fix column naming issue on import in vensim: add sheet name to column variables.
2. Check length of variables and their mapping --> buurten have more wijken as mapping than wijken have entities (see modelfix file)

## Step 1: importing data

In [2]:
'''Import Data'''

# BAG data
df = pd.read_csv('C:/Users/markhupkens/Dropbox/EnTransitionNL/0. Data/masterdf.csv', error_bad_lines=False, sep =';') # handled data from https://www.rvo.nl/sites/default/files/2019/01/Voorlopige_labels_okt2018.zip

# Energy Data
df_energy_buurt = pd.read_excel("C:/Users/markhupkens/Dropbox/EnTransitionNL/0. Data/Klimaatmonitor (energieverbruik , stadsverwarming, pv - Buurten,wijken en gemeenten 2017).xls",sheet_name='Buurt')
df_energy_gemeente = pd.read_excel("C:/Users/markhupkens/Dropbox/EnTransitionNL/0. Data/Klimaatmonitor (energieverbruik , stadsverwarming, pv - Buurten,wijken en gemeenten 2017).xls",sheet_name='Gemeente')
df_energy_wijk = pd.read_excel("C:/Users/markhupkens/Dropbox/EnTransitionNL/0. Data/Klimaatmonitor (energieverbruik , stadsverwarming, pv - Buurten,wijken en gemeenten 2017).xls",sheet_name='Wijk')
df_inkomen_buurt = pd.read_csv("C:/Users/markhupkens/Dropbox/EnTransitionNL/0. Data/CBS_2017_Inkomen_Buurt.csv",sep=';')

# Replace string values with nan value 0.424
df_energy_buurt.replace(to_replace='?',value=0.4242,inplace=True)
df_energy_buurt.replace(to_replace='-',value=0.4242,inplace=True)

df_energy_gemeente.replace(to_replace='?',value=0.4242,inplace=True)
df_energy_gemeente.replace(to_replace='-',value=0.4242,inplace=True)

df_energy_wijk.replace(to_replace='?',value=0.4242,inplace=True)
df_energy_wijk.replace(to_replace='-',value=0.4242,inplace=True)

# green gas and renewable heat
df_heat = pd.read_excel("C:/Users/markhupkens/Dropbox/EnTransitionNL/0. Data/Klimaatmonitor. Green gas and heat- Gemeenten.xls",sheet_name='Gemeenten')


### 1. Solar PV data

In [3]:
'''1. Solar PV per neighborhood'''

# Solar PV data, url: https://www.cbs.nl/nl-nl/nieuws/2019/17/vermogen-zonnepanelen-meer-dan-de-helft-toegenomen
df_solar = pd.read_csv("C:/Users/markhupkens/Dropbox/EnTransitionNL/0. Data/Zonnestroom__wijken_en_buurten__2017_16052019_130403.csv", sep=';', error_bad_lines=False)

# clean data
df_solar['Regioaanduiding/Soort regio (omschrijving)'] = df_solar['Regioaanduiding/Soort regio (omschrijving)'].str.split(" ").str[0]

# Select levels
df_solar_buurt = df_solar.loc[df_solar['Regioaanduiding/Soort regio (omschrijving)']=='Buurt']
df_solar_wijk = df_solar.loc[df_solar['Regioaanduiding/Soort regio (omschrijving)']=='Wijk']
df_solar_gem = df_solar.loc[df_solar['Regioaanduiding/Soort regio (omschrijving)']=='Gemeente']

In [4]:
# To Do

### 2. FEV and PHEV

In [5]:
''' FEV and PHEV Data'''

df_fev = pd.read_excel('C:/Users/markhupkens/Dropbox/EnTransitionNL/0. Data/Klimaatmonitor - Aantal geregistreerde  EV- PHEV2019 - Postcodes.xls',sheet_name='Auto s voertuigen 2019 Postcode')
df_voertuig = pd.read_csv('C:/Users/markhupkens/Dropbox/EnTransitionNL/0. Data/Motorvoertuigen_cbs_postcode.csv',error_bad_lines=False, sep=';')
df_pc = pd.read_csv('C:/Users/markhupkens/Dropbox/EnTransitionNL/0. Data/2017-cbs-pc6huisnr20170801_buurt/pc6hnr20170801_gwb.csv',sep=';')
df_pc_buurt = pd.read_csv('C:/Users/markhupkens/Dropbox/EnTransitionNL/0. Data/2017-cbs-pc6huisnr20170801_buurt/buurtnaam2017.csv',sep=';',encoding = 'unicode_escape')
df_pc_wijk = pd.read_csv('C:/Users/markhupkens/Dropbox/EnTransitionNL/0. Data/2017-cbs-pc6huisnr20170801_buurt/wijknaam2017.csv',sep=';',encoding = 'unicode_escape')
df_pc_gemeente = pd.read_csv('C:/Users/markhupkens/Dropbox/EnTransitionNL/0. Data/2017-cbs-pc6huisnr20170801_buurt/gemeentenaam2017.csv',sep=';',encoding = 'unicode_escape')

a. Zipcode coupling dataframe (CBS)

In [6]:
df_pc = df_pc.merge(df_pc_buurt, 
                    left_on='Buurt2017', 
                    right_on='BUURT2017').merge(df_pc_wijk, 
                                                left_on='Wijk2017', 
                                                right_on='WIJK2017').merge(df_pc_gemeente, 
                                                                           left_on='Gem2017', 
                                                                           right_on='GEM2017')

df_pc['PC4'] = df_pc['PC6'].str[:4]
df_pc.drop_duplicates(subset='PC4', keep='first', inplace=True)

b. Vehicles dataframe (CBS + Klimaatmonitor (RDW)

In [7]:
# Create merge columns
df_fev['Postcode'] = df_fev["Auto's/voertuigen 2019 - Postcodes"].str.split(" ").str[0]
df_voertuig['Postcode'] = df_voertuig['RegioS'].str.split(" ").str[0]
df_fev = df_fev.merge(df_voertuig,left_on='Postcode', right_on='Postcode', how='inner')

# Add buurt,wijk and gemeente names based on pc4
df_veh = df_fev.merge(df_pc, left_on='Postcode', right_on='PC4', how='inner')

c. group data

In [8]:
'''Group data in wijken en gemeenten (pc4)'''

# Gemeenten
df_veh_gem = df_veh.groupby('GEMNAAM').agg({"Aantal geregistreerde elektrische personenauto's (FEV)":'sum',
                                                                              "Aantal geregistreerde plug-in hybride personenauto's (PHEV)":'sum',
                                                                              "Aantal geregistreerde personenauto's op aardgas (CNG)":'sum',
                                                                              "Aantal geregistreerde hybride personenauto's":'sum',
                                                                              "Personenauto's op waterstof (H2)":'sum',
                                                                             'Personenauto_2':'sum'})
df_veh_gem['Benzine_Diesel']=df_veh_gem['Personenauto_2']-(df_veh_gem["Aantal geregistreerde elektrische personenauto's (FEV)"] + 
                                                           df_veh_gem["Aantal geregistreerde plug-in hybride personenauto's (PHEV)"] + 
                                                           df_veh_gem["Aantal geregistreerde personenauto's op aardgas (CNG)"] + 
                                                           df_veh_gem["Aantal geregistreerde hybride personenauto's"] + 
                                                           df_veh_gem["Personenauto's op waterstof (H2)"])
# Wijken
df_veh_wijk = df_veh.groupby('WIJKNAAM').agg({"Aantal geregistreerde elektrische personenauto's (FEV)":'sum',
                                                                              "Aantal geregistreerde plug-in hybride personenauto's (PHEV)":'sum',
                                                                              "Aantal geregistreerde personenauto's op aardgas (CNG)":'sum',
                                                                              "Aantal geregistreerde hybride personenauto's":'sum',
                                                                              "Personenauto's op waterstof (H2)":'sum',
                                                                             'Personenauto_2':'sum'})
df_veh_wijk['Benzine_Diesel']=df_veh_wijk['Personenauto_2']-(df_veh_wijk["Aantal geregistreerde elektrische personenauto's (FEV)"] + 
                                                           df_veh_wijk["Aantal geregistreerde plug-in hybride personenauto's (PHEV)"] + 
                                                           df_veh_wijk["Aantal geregistreerde personenauto's op aardgas (CNG)"] + 
                                                           df_veh_wijk["Aantal geregistreerde hybride personenauto's"] + 
                                                           df_veh_wijk["Personenauto's op waterstof (H2)"])



In [9]:
df_veh_wijk.drop('Personenauto_2', axis=1, inplace=True)
df_veh_gem.drop('Personenauto_2', axis=1, inplace=True)

In [10]:
df_veh_wijk.head()

Aantal geregistreerde elektrische personenauto's (FEV)  \
WIJKNAAM                                                                                                     
's Gravenmoer                                                                                     4.0        
's-Gravenpolder                                                                                   3.0        
's-Heer Abtskerke                                                                                 8.0        
Aalsmeerderbrug/ Oude Meer/ Rozenburg / Schipho...                                               89.0        
Aarlanderveen                                                                                     3.0        

                                                    Aantal geregistreerde plug-in hybride personenauto's (PHEV)  \
WIJKNAAM                                                                                                          
's Gravenmoer                                                                                     8.0             
's-Gravenpolder                                                                                  10.0             
's-Heer Abtskerke                                                                                17.0             
Aalsmeerderbrug/ Oude Meer/ Rozenburg / Schipho...                                               36.0             
Aarlanderveen                                                                                     4.0             

                                                    Aantal geregistreerde personenauto's op aardgas (CNG)  \
WIJKNAAM                                                                                                    
's Gravenmoer                                                                                     1.0       
's-Gravenpolder                                                                                   1.0       
's-Heer Abtskerke                                                                                 0.0       
Aalsmeerderbrug/ Oude Meer/ Rozenburg / Schipho...                                               46.0       
Aarlanderveen                                                                                     0.0       

                                                    Aantal geregistreerde hybride personenauto's  \
WIJKNAAM                                                                                           
's Gravenmoer                                                                               25.0   
's-Gravenpolder                                                                             38.0   
's-Heer Abtskerke                                                                           52.0   
Aalsmeerderbrug/ Oude Meer/ Rozenburg / Schipho...                                          20.0   
Aarlanderveen                                                                               26.0   

                                                    Personenauto's op waterstof (H2)  \
WIJKNAAM                                                                               
's Gravenmoer                                                                    0.0   
's-Gravenpolder                                                                  0.0   
's-Heer Abtskerke                                                                0.0   
Aalsmeerderbrug/ Oude Meer/ Rozenburg / Schipho...                               0.0   
Aarlanderveen                                                                    0.0   

                                                    Benzine_Diesel  
WIJKNAAM                                                            
's Gravenmoer                                               1364.0  
's-Gravenpolder                                             2502.0  
's-Heer Abtskerke                                           3080.0  
Aalsmeerderbrug/ Oude Meer/ Rozenburg / Schipho.

## Building stock data on buurt-level

Firstly, binary columns have to be created to allow for counting in a groupby dataframe. 13 new columns are added to show housing type (c1-c6) and label (A-G).

In [11]:
''' Create housing matrix'''

# Add housing matrix to enable building type count at the end of the script
df.loc[df['Housing Type']=='C1', 'Houses Detached BAG2018d'] = 1 
df.loc[df['Housing Type']=='C2', 'Houses 2u1Roof BAG2018d'] = 1 
df.loc[df['Housing Type']=='C3', 'Houses Corner BAG2018d'] = 1
df.loc[df['Housing Type']=='C4', 'Houses Row BAG2018'] = 1
df.loc[df['Housing Type']=='C5', 'Houses SingleFloorAppartments c5 BAG2018d'] = 1
df.loc[df['Housing Type']=='C6', 'Houses MultiFloorAppartments c6 BAG2018d'] = 1

# Add preliminary evaluation
df.loc[df['Preliminary Evaluation']=='A', 'Label A BAG2018d'] = 1 
df.loc[df['Preliminary Evaluation']=='B', 'Label B BAG2018d'] = 1 
df.loc[df['Preliminary Evaluation']=='C', 'Label C BAG2018d'] = 1
df.loc[df['Preliminary Evaluation']=='D', 'Label D BAG2018d'] = 1
df.loc[df['Preliminary Evaluation']=='E', 'Label E BAG2018d'] = 1
df.loc[df['Preliminary Evaluation']=='F', 'Label F BAG2018d'] = 1
df.loc[df['Preliminary Evaluation']=='G', 'Label G BAG2018d'] = 1

# Add Numerical labels (A=1, G=7)
df.loc[df['Preliminary Evaluation']=='A', 'Average Label BAG2018d'] = 1
df.loc[df['Preliminary Evaluation']=='B', 'Average Label BAG2018d'] = 2
df.loc[df['Preliminary Evaluation']=='C', 'Average Label BAG2018d'] = 3
df.loc[df['Preliminary Evaluation']=='D', 'Average Label BAG2018d'] = 4
df.loc[df['Preliminary Evaluation']=='E', 'Average Label BAG2018d'] = 5
df.loc[df['Preliminary Evaluation']=='F', 'Average Label BAG2018d'] = 6
df.loc[df['Preliminary Evaluation']=='G', 'Average Label BAG2018d'] = 7


# convert to string
df['Neighbourhood Code'] = df['Neighbourhood Code'].astype(str).str.split(".").str[0] #.map(str).str.split(".").str[0]
df['District Code'] = df['District Code'].astype(str).str.split(".").str[0] # or .map(str).str.split(".").str[0]

#### 1. Group Data

In [12]:
'''Group adressdata in neighborhoods within municipalities'''

# group data
df_bag = df.groupby(['Municipality Name',
                     'Neighbourhood Name',
                     'Neighbourhood Code']).agg({'House No':'count',
                                              'Houses Detached BAG2018d':'count',
                                               'Houses 2u1Roof BAG2018d':'count',
                                              'Houses Corner BAG2018d':'count',
                                              'Houses Row BAG2018':'count',
                                              'Houses SingleFloorAppartments c5 BAG2018d':'count',
                                              'Houses MultiFloorAppartments c6 BAG2018d':'count',
                                              'Construction Year':'mean',
                                               'Label A BAG2018d':'count',
                                               'Label B BAG2018d':'count',
                                                'Label C BAG2018d':'count',
                                                'Label D BAG2018d':'count',
                                                'Label E BAG2018d':'count',
                                                'Label F BAG2018d':'count',
                                                'Label G BAG2018d':'count',
                                                'Average Label BAG2018d':'mean'})

# Rename column and duplicate index for merge later on
df_bag.rename(columns={'House No':'Houses All BAG2018d'})
df_bag['Neighbourhood Name_2'] = df_bag.index.get_level_values('Neighbourhood Name')
df_bag['Municipality Name_2'] = df_bag.index.get_level_values('Municipality Name')
df_bag['Neighbourhood Code_2'] = df_bag.index.get_level_values('Neighbourhood Code') # string values for easy merge

In [13]:
df_bag['Average Label BAG2018d'].describe()

count    12987.000000
mean         4.093182
std          1.309312
min          1.000000
25%          3.149821
50%          4.166434
75%          5.048930
max          7.000000
Name: Average Label BAG2018d, dtype: float64

In [14]:
'''Group adressdata in districts within municipalities'''

# group data
df_bag_wijk = df.groupby(['Municipality Name',
                          'District Name',
                          'District Code']).agg({'House No':'count',
                                              'Houses Detached BAG2018d':'count',
                                               'Houses 2u1Roof BAG2018d':'count',
                                              'Houses Corner BAG2018d':'count',
                                              'Houses Row BAG2018':'count',
                                              'Houses SingleFloorAppartments c5 BAG2018d':'count',
                                              'Houses MultiFloorAppartments c6 BAG2018d':'count',
                                              'Construction Year':'mean',
                                               'Label A BAG2018d':'count',
                                               'Label B BAG2018d':'count',
                                                'Label C BAG2018d':'count',
                                                'Label D BAG2018d':'count',
                                                'Label E BAG2018d':'count',
                                                'Label F BAG2018d':'count',
                                                'Label G BAG2018d':'count'})

# Rename column and duplicate index for merge later on
df_bag_wijk.rename(columns={'House No':'Houses All BAG2018d'})
df_bag_wijk['District Name_2'] = df_bag_wijk.index.get_level_values('District Name')
df_bag_wijk['Municipality Name_2'] = df_bag_wijk.index.get_level_values('Municipality Name')
df_bag_wijk['District Code_2'] = df_bag_wijk.index.get_level_values('District Code') # string values for easy merge

In [15]:
'''Group adressdata in districts within municipalities'''

# group data
df_bag_gem = df.groupby(['Municipality Name']).agg({'House No':'count',
                                              'Houses Detached BAG2018d':'count',
                                               'Houses 2u1Roof BAG2018d':'count',
                                              'Houses Corner BAG2018d':'count',
                                              'Houses Row BAG2018':'count',
                                              'Houses SingleFloorAppartments c5 BAG2018d':'count',
                                              'Houses MultiFloorAppartments c6 BAG2018d':'count',
                                              'Construction Year':'mean',
                                               'Label A BAG2018d':'count',
                                               'Label B BAG2018d':'count',
                                                'Label C BAG2018d':'count',
                                                'Label D BAG2018d':'count',
                                                'Label E BAG2018d':'count',
                                                'Label F BAG2018d':'count',
                                                'Label G BAG2018d':'count'})

# Rename column and duplicate index for merge later on
df_bag_gem.rename(columns={'House No':'Houses All BAG2018d'})
df_bag_gem['Municipality Name_2'] = df_bag_gem.index.get_level_values('Municipality Name')

## Step 2 Merge All Data
- __df_bag[buurt, wijk, gemeente]__: data for each building on housingtype, provisional label and building year
- __df_energy [buurt, wijk, gemeente]__: average energy consumption
- __df_solar[buurt, wijk, gemeente]__: # installations and KW
- __df_veh[wijk, gemeente]__: EV's, PHEV's and Diesel/petrol

### Buurten

In [16]:
df_inkomen_buurt.head()

ID WijkenEnBuurten                            Gemeentenaam_1  Codering_3  \
0   0      NL00        Nederland                                 NL00         
1   1      GM1680      Aa en Hunze                               GM1680       
2   2      WK168000    Aa en Hunze                               WK168000     
3   3      BU16800000  Aa en Hunze                               BU16800000   
4   4      BU16800009  Aa en Hunze                               BU16800009   

  GemiddeldInkomenPerInkomensontvanger_65 GemiddeldInkomenPerInwoner_66  
0                                    32.0                          25.6  
1                                    31.6                          25.9  
2                                    34.0                          27.5  
3                                    33.3                          26.8  
4                                       .                             .

In [17]:
'''Buurt: merge grouped bag data with klimaatmonitordata on buurt'''

df_merged_buurt = df_bag.merge(df_energy_buurt,
                               left_on='Neighbourhood Name_2', 
                               right_on='Buurt',how='left').merge(df_solar_buurt, 
                                                                   left_on='Neighbourhood Name_2',
                                                                   right_on=df_solar_buurt['Wijken en buurten'],
                                                                  how='left')

df_merged_buurt = df_merged_buurt.groupby(['Municipality Name_2',
                                           'Neighbourhood Name_2',
                                           'Neighbourhood Code_2']).mean() # group by original index

### Wijken

In [18]:
df_veh_wijk.index

Index([''s Gravenmoer', ''s-Gravenpolder', ''s-Heer Abtskerke',
       'Aalsmeerderbrug/ Oude Meer/ Rozenburg / Schiphol Rijk',
       'Aarlanderveen', 'Abbenes / Buitenkaag', 'Abcoude', 'Achthuizen',
       'Aetsveldsepolder', 'Afferden',
       ...
       'Zuid', 'Zuidas', 'Zuidland', 'Zuidoost', 'Zuidwest', 'Zuigerplaspark',
       'Zwaanshoek', 'Zwammerdam', 'Zwanenburg', 'de Hoef'],
      dtype='object', name='WIJKNAAM', length=1903)

In [19]:
'''Wijk merge grouped bag data with klimaatmonitordata on wijk'''

df_merged_wijk = df_bag_wijk.merge(df_energy_wijk,
                                   left_on='District Name', 
                                   right_on='Wijk',
                                   how='left').merge(df_veh_wijk,
                                                    left_on='District Name_2', 
                                                    right_index=True,
                                                    how='left').merge(df_solar_wijk, 
                                                                       left_on='District Name_2',
                                                                       right_on='Wijken en buurten',
                                                                       how='left') 

df_merged_wijk = df_merged_wijk.groupby(['Municipality Name_2',
                                         'District Name_2',
                                         'District Code_2']).mean() # group by original index

### Gemeenten

In [20]:
'''Gemeente merge grouped bag data with klimaatmonitordata on Gemeente'''

df_merged_gemeente = df_bag_gem.merge(df_energy_gemeente,
                                  left_on='Municipality Name', 
                                  right_on='Gemeente',
                                  how='left').merge(df_veh_gem,
                                                    left_on='Municipality Name_2', 
                                                    right_index=True,
                                                    how='left').merge(df_heat, 
                                                                      left_on='Municipality Name_2', 
                                                                      right_on="Thema's  - Gemeenten",
                                                                     how='left')

df_merged_gemeente = df_merged_gemeente.merge(df_solar_gem, 
                                              left_on='Municipality Name_2', # .astype(str),
                                              right_on='Wijken en buurten', #.str[2:].str.strip(" ").str.lstrip("0"),
                                              how='left') 

df_merged_gemeente = df_merged_gemeente.groupby(['Municipality Name_2']).mean() # group by original index

#### Clean merged dataset

In [21]:
# Drop index columns (used for merging)
l_drop= ['Huisnummer', 'Buurt2017', 'Wijk2017', 'Gem2017', 'BUURT2017',
       'WIJK2017', 'GEM2017']

# df_merged_buurt.drop(l_drop,axis=1,inplace=True)
# df_merged_gemeente.drop(l_drop,axis=1,inplace=True)
# df_merged_wijk.drop(l_drop,axis=1,inplace=True)

In [22]:
# Remove special characters from column names
df_merged_gemeente.columns = df_merged_gemeente.columns.str.replace("[","").str.replace("]","")
df_merged_buurt.columns = df_merged_buurt.columns.str.replace("[","").str.replace("]","")
df_merged_wijk.columns = df_merged_wijk.columns.str.replace("[","").str.replace("]","")

## Step 3. Create new Model-setup
Import modelsetup file and merge new data on geospatial index (buurt, wijk, gemeente)

In [23]:
# Import model setup files

df_mod_gemeente = pd.read_excel("C:/users/markhupkens/Dropbox/EnTransitionNL/0. Data/ModelSetUpEnergieNL02 (1).xlsx",sheet_name='gemeente')
df_mod_buurt = pd.read_excel("C:/users/markhupkens/Dropbox/EnTransitionNL/0. Data/ModelSetUpEnergieNL02 (1).xlsx",sheet_name='buurt')
df_mod_wijk = pd.read_excel("C:/users/markhupkens/Dropbox/EnTransitionNL/0. Data/ModelSetUpEnergieNL02 (1).xlsx",sheet_name='wijk') # wijkdata horrible from klimaatmonitor
df_mod_mod = pd.read_excel("C:/users/markhupkens/Dropbox/EnTransitionNL/0. Data/ModelSetUpEnergieNL02 (1).xlsx",sheet_name='ModelSpecification')

In [24]:
# split entity string to match building data on municipality name
df_mod_gemeente['Municipality Name'] = df_mod_gemeente["Entities"].str.split(" G").str[0] 
df_mod_wijk['Wijk Code'] = df_mod_wijk["Entities"].str.split(" W").str[-1].str.strip("K").str.strip(" ").str.lstrip("0") # Wijk on wijkcode, ditching leading 0's
df_mod_buurt['Buurt Code'] = df_mod_buurt["Entities"].str.split(" B").str[-1].str.strip("U").str.strip(" ").str.lstrip("0") # Buurt on buurt code, ditching leading 0's

In [25]:
# fillna with 0.4242

df_merged_gemeente.fillna(0.4242,inplace=True)
df_merged_wijk.fillna(0.4242,inplace=True)
df_merged_buurt.fillna(0.4242,inplace=True)

In [26]:
'''Merge prepared data with modelsetup data'''

# Gemeenten
df_mod_gemeente = df_mod_gemeente.merge(df_merged_gemeente, left_on='Municipality Name', right_on= df_merged_gemeente.index,how='inner')
df_mod_gemeente = df_mod_gemeente.drop(['Municipality Name'], axis=1)

# Buurten
df_mod_buurt = df_mod_buurt.merge(df_merged_buurt, 
                                  left_on='Buurt Code', 
                                  right_on= df_merged_buurt.index.get_level_values('Neighbourhood Code_2'),
                                  how='inner')
                                                     # df_mod_buurt = df_mod_buurt.drop(['Buurt Name'], axis=1)

# Wijken
df_mod_wijk = df_mod_wijk.merge(df_merged_wijk, left_on='Wijk Code', right_on=df_merged_wijk.index.get_level_values('District Code_2'),how='inner')
# df_mod_wijk = df_mod_wijk.drop(['District Name_2'], axis=1)

### Add classes to neighbourhoods 
- based on building value, assign value classes. divide in quantiles and assign integer
- based on % district heat, assign quantitles of district heat availability

In [27]:
'''Add quantile groups of average labels''' # LOW QUANTILE = GOOD LABEL (A=1)

df_mod_buurt.loc[df_mod_buurt['Average Label BAG2018d']<=df_mod_buurt['Average Label BAG2018d'].quantile(q=0.25), 'label group'] = 1 

df_mod_buurt.loc[(df_mod_buurt['Average Label BAG2018d']>=df_mod_buurt['Average Label BAG2018d'].quantile(q=0.25)) &
                 (df_mod_buurt['Average Label BAG2018d']<=df_mod_buurt['Average Label BAG2018d'].quantile(q=0.50)), 'label group'] = 2 

df_mod_buurt.loc[(df_mod_buurt['Average Label BAG2018d']>=df_mod_buurt['Average Label BAG2018d'].quantile(q=0.50)) &
                 (df_mod_buurt['Average Label BAG2018d']<=df_mod_buurt['Average Label BAG2018d'].quantile(q=0.75)), 'label group'] = 3 

df_mod_buurt.loc[(df_mod_buurt['Average Label BAG2018d']>=df_mod_buurt['Average Label BAG2018d'].quantile(q=0.75)) &
                 (df_mod_buurt['Average Label BAG2018d']<=df_mod_buurt['Average Label BAG2018d'].quantile(q=1.00)), 'label group'] = 4 

In [28]:
df_mod_buurt['label group'].value_counts()

1.0    3247
4.0    3247
3.0    3247
2.0    3247
Name: label group, dtype: int64

In [29]:
'''Add value classes'''
# distribute neighourbhoods in 4 classes based on building value from 1 (lowest quantile) to 4 (highest quantile)

df_mod_buurt.loc[df_mod_buurt['woningwaarde keuro buurt']<=df_mod_buurt['woningwaarde keuro buurt'].quantile(q=0.25), 'value group'] = 1 

df_mod_buurt.loc[(df_mod_buurt['woningwaarde keuro buurt']>=df_mod_buurt['woningwaarde keuro buurt'].quantile(q=0.25)) &
                 (df_mod_buurt['woningwaarde keuro buurt']<=df_mod_buurt['woningwaarde keuro buurt'].quantile(q=0.50)), 'value group'] = 2 

df_mod_buurt.loc[(df_mod_buurt['woningwaarde keuro buurt']>=df_mod_buurt['woningwaarde keuro buurt'].quantile(q=0.50)) &
                 (df_mod_buurt['woningwaarde keuro buurt']<=df_mod_buurt['woningwaarde keuro buurt'].quantile(q=0.75)), 'value group'] = 3 

df_mod_buurt.loc[(df_mod_buurt['woningwaarde keuro buurt']>=df_mod_buurt['woningwaarde keuro buurt'].quantile(q=0.75)) &
                 (df_mod_buurt['woningwaarde keuro buurt']<=df_mod_buurt['woningwaarde keuro buurt'].quantile(q=1.00)), 'value group'] = 4 

In [30]:
df_mod_buurt['value group'].value_counts()

4.0    3264
3.0    3258
1.0    3236
2.0    3230
Name: value group, dtype: int64

In [31]:
df_show = df_mod_buurt.drop(columns=['Bevolking buurt', 'Mannen buurt',
       'Vrouwen buurt', 'Bevolking 0 tot 15 jaar buurt',
       'Bevolking 15 tot 25 jaar buurt', 'Bevolking 25 tot 45 jaar buurt',
       'Bevolking 45 tot 65 jaar buurt', 'Bevolking 65 jaar of ouder buurt',
       'Bevolking Geboorte en sterfte Geboorte totaal aantal buurt',
       'Bevolking Geboorte en sterfte Geboorte relatief per 1 000 inwoners buurt',
       'Bevolking Geboorte en sterfte Sterfte totaal aantal buurt',
       'Bevolking Geboorte en sterfte Sterfte relatief per 1 000 inwoners buurt',
       'Huishoudens totaal buurt', 'Eenpersoonshuishoudens buurt',
       'Huishoudens zonder kinderen buurt', 'Huishoudens met kinderen buurt',
       'huishoudensgrootte ', 'Bevolkingsdichtheid per sqkm buurt',
       'Woningvoorraad buurt', 'woningwaarde keuro buurt',
       'eengezinswoning pc buurt', 'meergezinswoning pc buurt',
       'onbewoond pc buurt', 'Koopwoningen pc buurt',
       'Huurwoningen totaal pc buurt',
       'Huurwoningen woningcorporatie pc buurt',
       'Huurwoningen overige verhuurders pc buurt',
       'Eigendom onbekend pc buurt', 'Bouwjaar voor 2000 pc buurt',
       'Oppervlakte ', 'Oppervlakte land ha buurt',
       'Oppervlakte water ha buurt', 'Buurt Code', 'House No'])

df_show.head()

Entities                   Mapping  \
0                    Annen BU16800000  Wijk 00 Annen WK168000     
1  Verspreide huizen Annen BU16800009  Wijk 00 Annen WK168000     
2                     Eext BU16800100   Wijk 01 Eext WK168001     
3   Verspreide huizen Eext BU16800109   Wijk 01 Eext WK168001     
4                    Anloo BU16800200  Wijk 02 Anloo WK168002     

   Houses Detached BAG2018d  Houses 2u1Roof BAG2018d  Houses Corner BAG2018d  \
0                       582                      478                     160   
1                        61                        1                       0   
2                       301                      146                      37   
3                        47                        1                       0   
4                        99                       41                       2   

   Houses Row BAG2018  Houses SingleFloorAppartments c5 BAG2018d  \
0                 204                                        134   
1                   0                                          0   
2                  40                                         16   
3                   0                                          1   
4                   1                                          2   

   Houses MultiFloorAppartments c6 BAG2018d  Construction Year  \
0                                         0        1973.298460   
1                                         0        1956.096774   
2                                         0        1954.994444   
3                                         0        1946.428571   
4                                         0        1954.496552   

   Label A BAG2018d  ...  Gemiddeld elektriciteitsgebruik hoekwoning kWh 2017  \
0                72  ...                                          2860.0000     
1                 3  ...                                             0.4242     
2                11  ...                                          2440.0000     
3                 1  ...                                             0.4242     
4                 4  ...                                             0.4242     

   Gemiddeld elektriciteitsgebruik twee-onder-een-kap-woning kWh 2017  \
0                                          3300.0000                    
1                                             0.4242                    
2                                          3150.0000                    
3                                             0.4242                    
4                                          2600.0000                    

   Gemiddeld elektriciteitsgebruik tussenwoning kWh 2017  \
0                                          2480.0000       
1                                             0.4242       
2                                          1970.0000       
3                                             0.4242       
4                                             0.4242       

   Gemiddeld elektriciteitsgebruik vrijstaande woning kWh 2017  \
0                                             3720.0             
1                                             4830.0             
2                                             3590.0             
3                                             3340.0             
4                                             3760.0             

   Aandeel stadsverwarming % 2017  \
0                             0.0   
1                             0.0   
2                             0.0   
3                             0.0   
4                             0.0   

   Opgesteld vermogen geregistreerde PV-panelen per adres/locatie (gemiddelde) kWpiek 2015  \
0                                                7.0                                         
1                                                7.0                                         
2                                                4.0                                         
3                                                6.0    

#### Check data on income per neighbourhood

In [32]:
df_inkomen_buurt = pd.read_csv("C:/Users/markhupkens/Dropbox/EnTransitionNL/0. Data/CBS_2017_Inkomen_Buurt.csv", sep=';')

df_inkomen_buurt['GemiddeldInkomenPerInkomensontvanger_65'] = pd.to_numeric(df_inkomen_buurt['GemiddeldInkomenPerInkomensontvanger_65'].str.lstrip(' ').replace('.', np.nan)) * 1000
df_inkomen_buurt['GemiddeldInkomenPerInwoner_66'] = pd.to_numeric(df_inkomen_buurt['GemiddeldInkomenPerInwoner_66'].str.lstrip(' ').replace('.', np.nan)) * 1000

len(df_inkomen_buurt.loc[df_inkomen_buurt['GemiddeldInkomenPerInwoner_66'] >0])

4072

In [33]:
len(df_mod_buurt.loc[df_mod_buurt['woningwaarde keuro buurt']>0]) - len(df_mod_buurt.loc[df_mod_buurt['woningwaarde keuro buurt']==0.4242])

10099

Data completeness of building value is far higher than data completeness of income per capita or average income per person.

### 4. Export new modelspecification file

In [34]:
# export as xlsx to genereate new modelspecification file
from pandas import ExcelWriter

# with pd.ExcelWriter('D:/markhupkens/Dropbox/EnTransitionNL/0. Data/ModelSetUpEnergyNL01_MH.xlsx') as writer:  # doctest: +SKIP
with pd.ExcelWriter('C:/Users/markhupkens/Dropbox/EnTransitionNL/ModelSetUpEnergyNL01_MH_avglabelgroup.xlsx') as writer:  # doctest: +SKIP
    df_mod_buurt.to_excel(writer, sheet_name='buurt')
    df_mod_wijk.to_excel(writer, sheet_name='wijk')
    df_mod_gemeente.to_excel(writer, sheet_name='gemeente')
    df_mod_mod.to_excel(writer, sheet_name='ModelSpecification')

## OR ADD SPECIFIC COLUMN TO EXISTING FILE

In [35]:
# Select columsn to merge
df_new = df_mod_buurt[['Entities', 'label group','Average Label BAG2018d']]

In [36]:
# Import exsiting file
df_import_gemeente = pd.read_excel("C:/users/markhupkens/Dropbox/EnTransitionNL/MSETMnlEPdataMHv02.xlsx",sheet_name='gemeente')
df_import_buurt = pd.read_excel("C:/users/markhupkens/Dropbox/EnTransitionNL/MSETMnlEPdataMHv02.xlsx",sheet_name='buurt')
df_import_wijk = pd.read_excel("C:/users/markhupkens/Dropbox/EnTransitionNL/MSETMnlEPdataMHv02.xlsx",sheet_name='wijk') # wijkdata horrible from klimaatmonitor
df_import_mod = pd.read_excel("C:/users/markhupkens/Dropbox/EnTransitionNL/MSETMnlEPdataMHv02.xlsx",sheet_name='ModelSpecification')

In [37]:
df_final_buurt = df_import_buurt.merge(df_new,left_on='Entities', right_on='Entities',how='inner')

# export as xlsx to genereate new modelspecification file
from pandas import ExcelWriter

# with pd.ExcelWriter('D:/markhupkens/Dropbox/EnTransitionNL/0. Data/ModelSetUpEnergyNL01_MH.xlsx') as writer:  # doctest: +SKIP
with pd.ExcelWriter('C:/Users/markhupkens/Dropbox/EnTransitionNL/ModelSetUpEnergyNL02_MH_avglabelgroup.xlsx') as writer:  # doctest: +SKIP
    df_final_buurt.to_excel(writer, sheet_name='buurt')
    df_import_wijk.to_excel(writer, sheet_name='wijk')
    df_import_gemeente.to_excel(writer, sheet_name='gemeente')
    df_import_mod.to_excel(writer, sheet_name='ModelSpecification')